In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-126796")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-126796
Azure region: southcentralus
Subscription id: a0f586ec-8016-4ea9-8248-9bf2299ad437
Resource group: aml-quickstarts-126796


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

vm_size = 'Standard_D2_V2'

try:
    compute_cluster = ComputeTarget(ws,'Avinash-ML1')
    print('found exisitng cluster, use it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = vm_size,max_nodes = 4)
    
    compute_cluster = ComputeTarget.create(ws,'Avinash-ML2',compute_config)

compute_cluster.wait_for_completion(show_output = True)


found exisitng cluster, use it

Running


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
    "--C" : uniform(0.7,5),
    "--max_iter" : uniform(20,60)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 1, slack_factor = 0.2, delay_evaluation = 5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = './',compute_target = compute_cluster, entry_script = 'train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est,
                                    hyperparameter_sampling = ps,
                                    primary_metric_name = 'accuracy',
                                    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs = 8,
                                    max_concurrent_runs = 4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

run  = exp.submit(hyperdrive_config)

RunDetails(run).show()

run.get_status()
run.wait_for_completion()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_035e433f-2fd1-47d3-984e-6e346e7f2cba',
 'target': 'Avinash-ML1',
 'status': 'Canceled',
 'startTimeUtc': '2020-11-18T17:50:23.976465Z',
 'endTimeUtc': '2020-11-18T17:58:57.002323Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": {\n      "type": null,\n      "message": "[(https://southcentralus.experiments.azureml.net/subscriptions/a0f586ec-8016-4ea9-8248-9bf2299ad437/resourceGroups/aml-quickstarts-126796/providers/Microsoft.Machin

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = run.get_best_run_by_primary_metric()
best_run_metrics = run.get_metrics()

param_values = best_run.get_details()['runDefinition']['arguments']







In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds1 = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [6]:
from train import clean_data

import pandas as pd
# Use the clean_data function to clean your data.
x, y = clean_data(ds1)


df_train = pd.concat([x,y],axis = 1)

In [7]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=df_train,
    label_column_name='y',
    n_cross_validations=4)

In [8]:
# Submit your automl run

### YOUR CODE HERE ###

run_automl = exp.submit(automl_config,show_output = True)

run_automl

run_automl.wait_for_completion()

Running on local machine
Parent Run ID: AutoML_f54e9f85-7703-47ca-ad82-c7010edfbe3e

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

{'runId': 'AutoML_f54e9f85-7703-47ca-ad82-c7010edfbe3e',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-11-18T18:07:48.795346Z',
 'endTimeUtc': '2020-11-18T18:39:58.805538Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"a0f586ec-8016-4ea9-8248-9bf2299ad437","resource_group":"aml-quickstarts-126796","workspace_name":"quick-starts-ws-126796","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":4,"y_min":null,"y

In [13]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_run, fitted_model = run_automl.get_output()
fitted_model.steps

best_run.register_model(model_path = './outputs/bestmodel.pkl',model_name = 'BestAutoML_voting_model')


ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/bestmodel.pkl in the set of files uploaded to the run: ['accuracy_table', 'confusion_matrix', 'explanation/35510d50/classes.interpret.json', 'explanation/35510d50/eval_data_viz.interpret.json', 'explanation/35510d50/expected_values.interpret.json', 'explanation/35510d50/features.interpret.json', 'explanation/35510d50/global_names/0.interpret.json', 'explanation/35510d50/global_rank/0.interpret.json', 'explanation/35510d50/global_values/0.interpret.json', 'explanation/35510d50/local_importance_values.interpret.json', 'explanation/35510d50/per_class_names/0.interpret.json', 'explanation/35510d50/per_class_rank/0.interpret.json', 'explanation/35510d50/per_class_values/0.interpret.json', 'explanation/35510d50/rich_metadata.interpret.json', 'explanation/35510d50/visualization_dict.interpret.json', 'explanation/f3956a3c/classes.interpret.json', 'explanation/f3956a3c/expected_val

In [11]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_f54e9f85-7703-47ca-ad82-c7010edfbe3e_34,,Completed,Link to Azure Machine Learning studio,Link to Documentation
